In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Telecom Customer Churn Prediction\n",
    "\n",
    "This notebook implements the data processing, modeling, and evaluation pipeline for the Customer Churn Prediction task."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "from sklearn.model_selection import train_test_split\n",
    "from sklearn.ensemble import RandomForestClassifier\n",
    "from sklearn.metrics import f1_score, roc_auc_score"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Data Loading"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Try loading from 'data/' directory (standard structure) or root (colab/local fallback)\n",
    "try:\n",
    "    df_billing = pd.read_csv(\"data/customer_billing_churn.csv\")\n",
    "    df_profile = pd.read_csv(\"data/customer_profile.csv\")\n",
    "except FileNotFoundError:\n",
    "    df_billing = pd.read_csv(\"customer_billing_churn.csv\")\n",
    "    df_profile = pd.read_csv(\"customer_profile.csv\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Data Preparation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Convert TotalCharges to numeric, coerce errors to NaN, fill with 0\n",
    "df_billing['TotalCharges'] = pd.to_numeric(df_billing['TotalCharges'], errors='coerce').fillna(0)\n",
    "\n",
    "# Constants\n",
    "EPS = 1e-6\n",
    "\n",
    "# Feature Engineering\n",
    "df_billing['AvgMonthlySpend'] = df_billing['TotalCharges'] / (df_billing['tenure'] + EPS)\n",
    "df_billing['TenureChargeInteraction'] = df_billing['MonthlyCharges'] * df_billing['tenure']\n",
    "\n",
    "# Tenure Stability Indicator\n",
    "df_billing['ShortTenureFlag'] = df_billing['tenure'].apply(lambda x: 1 if x < 12 else 0)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Dataset Integration & Encoding"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Combine on customerID\n",
    "df_merged = pd.merge(df_billing, df_profile, on='customerID')\n",
    "\n",
    "# Encode Churn (1 = Yes, 0 = No)\n",
    "df_merged['Churn'] = df_merged['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)\n",
    "\n",
    "# Identify categorical columns (excluding customerID and Churn)\n",
    "cat_cols = df_merged.select_dtypes(include=['object']).columns.tolist()\n",
    "if 'customerID' in cat_cols:\n",
    "    cat_cols.remove('customerID')\n",
    "\n",
    "# One-hot encode categorical variables\n",
    "churn_model_input = pd.get_dummies(df_merged, columns=cat_cols, drop_first=False, dtype=int)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Feature Selection & Splitting"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Drop customerID\n",
    "if 'customerID' in churn_model_input.columns:\n",
    "    churn_model_input = churn_model_input.drop(columns=['customerID'])\n",
    "\n",
    "# Ensure no missing values\n",
    "churn_model_input = churn_model_input.dropna()\n",
    "\n",
    "# Define X and y\n",
    "X = churn_model_input.drop(columns=['Churn'])\n",
    "y = churn_model_input['Churn']\n",
    "\n",
    "# Split dataset (70/30, seed 42)\n",
    "X_train, X_test, y_train, y_test = train_test_split(\n",
    "    X, y, test_size=0.30, random_state=42\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Model Training & Evaluation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "clf = RandomForestClassifier(random_state=42)\n",
    "clf.fit(X_train, y_train)\n",
    "\n",
    "# Predictions\n",
    "y_pred = clf.predict(X_test)\n",
    "y_prob = clf.predict_proba(X_test)[:, 1]\n",
    "\n",
    "# Metrics\n",
    "f1 = f1_score(y_test, y_pred)\n",
    "auc = roc_auc_score(y_test, y_prob)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Deliverables"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 1. Feature Importance Dictionary (Python Dict)\n",
    "importances = clf.feature_importances_\n",
    "feature_names = X.columns\n",
    "feature_importance_dict = {\n",
    "    feat: round(imp, 5) \n",
    "    for feat, imp in zip(feature_names, importances)\n",
    "}\n",
    "\n",
    "# 2. Model Metrics (Python Dict)\n",
    "model_metrics = {\n",
    "    \"f1_score\": round(f1, 5),\n",
    "    \"auc_score\": round(auc, 5)\n",
    "}\n",
    "\n",
    "# 3. Churn Counts (Serialized DataFrame)\n",
    "unique, counts = np.unique(y_pred, return_counts=True)\n",
    "churn_counts_df = pd.DataFrame({'class': unique, 'count': counts})\n",
    "churn_counts = churn_counts_df.to_dict(orient='split')\n",
    "\n",
    "# Output for verification\n",
    "print(\"Model Metrics:\", model_metrics)\n",
    "print(\"Churn Counts:\", churn_counts)"
   ]
  }
 ],
 "metadata": {
  "colab": {
   "provenance": []
  },
  "kernelspec": {
   "display_name": "Python 3",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.12"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 0
}